### Mecab 설치

In [1]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [3]:
cd Mecab-ko-for-Google-Colab/

[Errno 2] No such file or directory: 'Mecab-ko-for-Google-Colab/'
/content/Mecab-ko-for-Google-Colab


In [4]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
     |████████████████████████████████| 460kB 43.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-02-14 15:02:20--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c3:9b0a, 2406:da00:ff00::34cc:ea4a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downlo

### import 및 setting

In [16]:
from keras.preprocessing import sequence
from keras.models import load_model

import pickle, json, os
import re

from konlpy.tag import Mecab
mecab = Mecab()

# model load
model = load_model('/content/drive/MyDrive/STT_/model/Naver_dict_l3stm.h5')

# tokenizer load
with open("/content/drive/MyDrive/STT_/tokenizer/tokenizer3.pickle", 'rb') as fp:
    tokenizer = pickle.load(fp)

# token load
with open("/content/drive/MyDrive/STT_/token/token3.json", "r") as st_json:
    topic = json.load(st_json)

# stop words load
path ='/content/drive/MyDrive/STT_/korean_stopwords.txt'
f = open(path, 'r', encoding='utf-8')
stop_words = []
while True:
    line = f.readline()
    if not line: break
    stop_words.append(line.split()[0])
f.close()

max_word =20000
max_len = 1000 #max_len = 500

In [17]:
topic

{'0': '곤충',
 '1': '행정_기관_단체',
 '10': '만화',
 '11': '사회복지',
 '12': '음악용어',
 '13': '직업',
 '14': '아프리카',
 '15': '독일문학',
 '16': '무역',
 '17': '건축물',
 '18': '육아',
 '19': '북아메리카',
 '2': '한국_철학_윤리',
 '20': '건설_토목',
 '21': '농학',
 '22': '프랑스문학',
 '23': '영미문학',
 '24': '중동_역사_문화',
 '25': '한국_민속',
 '26': '무기',
 '27': '속담',
 '28': '기계_자동차_금속',
 '29': '국제기구',
 '3': '기타종교',
 '30': '외교',
 '31': '이슬람교',
 '32': '패션',
 '33': '경제',
 '34': '댄스_무용',
 '35': '동물',
 '36': '주류',
 '37': '한국_역사_문화',
 '38': '생명과학',
 '39': '호텔',
 '4': '커피_차',
 '40': '법',
 '41': '북한문학',
 '42': '아시아',
 '43': '남아메리카',
 '44': '헤어',
 '45': '서양_철학_윤리',
 '46': '불교',
 '47': '정치',
 '48': '재료',
 '49': '교양영어사전',
 '5': '클래식',
 '50': '명화',
 '51': '심리',
 '52': '화학',
 '53': '수학',
 '54': '물리학',
 '55': '세계_역사_문화',
 '56': '세계음식',
 '57': '중국문학',
 '58': '미술',
 '59': '한국음식',
 '6': '영화',
 '60': '의학',
 '61': '환경',
 '62': '색채',
 '63': '드라마',
 '64': '한국문학',
 '65': '아시아_역사_문화',
 '66': '교양_인문',
 '67': '질병',
 '68': '천문학',
 '69': '뮤지컬',
 '7': '군사',
 '70': '식물',


### PreProcessing

In [18]:
str_ = u'''삼위일체의 부정[편집] 기독교에서는 삼위일체 논리에 따라 예수 그리스도를 하나님의 아들이자 하나님의 구성요소로 보고 있으나, 
이슬람은 이를 기독교인들이 하나님을 신성모독하고 아브라함 계통의 종교에서 가장 금기시하는 우상숭배를 하는 것으로 간주한다.
'''

In [19]:
str_ = re.sub('[^A-Za-z0-9가-힣.]', ' ', str_)
str_ = re.sub(' +', ' ', str_)

In [20]:
temp = mecab.nouns(str_)
temp = [i for i in temp if i not in stop_words and i != '.']

In [23]:
sequences = tokenizer.texts_to_sequences([temp])
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
topic_idx = model.predict_classes(sequences_matrix)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
topic[str(topic_idx[0])]

'기독교'